# Práctica 2.A Toma de contacto con AIMA

La práctica está organizada en 3 partes. En la primera se muestra a través de ejemplos cómo se implementan algunos problemas clásicos como el de las jarras o el problema del ocho puzzle. En la segunda parte se muestra el uso de los algoritmos de búsqueda. En la tercera parte aprenderemos a medir las propiedades de los algoritmos.
En el notebook encontraras claramente identificados los lugares en los que debes incluir código o comentarios. 

## Parte I: Representación de problemas de espacios de estados.

### El primer paso es importar el código que necesitamos de search.py de AIMA y usar la clase Problem. En esta parte en vez de importarla la hemos copiado aquí para la explicación.

Como hemos visto en clase la representación de un problema de espacio de estados consiste en:
* Representar estados y acciones mediante una estructura de datos.
* Definir: estado_inicial, es_estado_final(_), acciones(_), aplica(_,_) y
  coste_de_aplicar_accion, si el problema tiene coste.

 La siguiente clase Problem representa este esquema general de cualquier
 problema de espacio de estados. Un problema concreto será una subclase de
 Problema, y requerirá implementar acciones, aplica y eventualmente __init__, actions,
 goal_test. La función coste_de_aplicar_accion la hemos incluido nosotros.

In [1]:
from search import *

In [2]:
   class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

    def coste_de_aplicar_accion(self, estado, accion):
        """Hemos incluido está función que devuelve el coste de un único operador (aplicar accion a estado). Por defecto, este
        coste es 1. Reimplementar si el problema define otro coste """ 
        return 1

Ahora vamos a ver un ejemplo de cómo definir un problema como subclase
de problema. En concreto, el problema de las jarras, visto en clase que es muy sencillo. 

In [3]:
class Jarras(Problem):
    """Problema de las jarras:
    Representaremos los estados como tuplas (x,y) de dos números enteros,
    donde x es el número de litros de la jarra de 4 e y es el número de litros
    de la jarra de 3"""

    def __init__(self):
        self.initial = (0,0)

    def actions(self,estado):
        jarra_de_4=estado[0]
        jarra_de_3=estado[1]
        accs=list()
        if jarra_de_4 > 0:
            accs.append("vaciar jarra de 4")
            if jarra_de_3 < 3:
                accs.append("trasvasar de jarra de 4 a jarra de 3")
        if jarra_de_4 < 4:
            accs.append("llenar jarra de 4")
            if jarra_de_3 > 0:
                accs.append("trasvasar de jarra de 3 a jarra de 4")
        if jarra_de_3 > 0:
            accs.append("vaciar jarra de 3")
        if jarra_de_3 < 3:
            accs.append("llenar jarra de 3")
        return accs

    def result(self,estado,accion):
        j4=estado[0]
        j3=estado[1]
        if accion=="llenar jarra de 4":
            return (4,j3)
        elif accion=="llenar jarra de 3":
            return (j4,3)
        elif accion=="vaciar jarra de 4":
            return (0,j3)
        elif accion=="vaciar jarra de 3":
            return (j4,0)
        elif accion=="trasvasar de jarra de 4 a jarra de 3":
            return (j4-3+j3,3) if j3+j4 >= 3 else (0,j3+j4)
        else: #  "trasvasar de jarra de 3 a jarra de 4"
            return (j3+j4,0) if j3+j4 <= 4 else (4,j3-4+j4)

    def goal_test(self,estado):
        return estado[0]==2


Vamos a probar algunos ejemplos.

In [4]:
p =Jarras()
p.initial


(0, 0)

In [5]:
p.actions(p.initial)

['llenar jarra de 4', 'llenar jarra de 3']

In [6]:
p.result(p.initial,"llenar jarra de 4")

(4, 0)

In [7]:
p.coste_de_aplicar_accion(p.initial,"llenar jarra de 4")

1

In [8]:
p.goal_test(p.initial)

False

# Representa el problema del puzzle de 8

### 8 Puzzle 

Tablero 3x3 cuyo objetivo es mover la configuración de las piezas desde un estado inicial dado a un estado objetivo moviendo las fichas al espacio en blanco. 

ejemplo:- 

                  Inicial                             Goal 
              | 7 | 2 | 4 |                       | 1 | 2 | 3 |
              | 5 | 0 | 6 |                       | 4 | 5 | 6 |
              | 8 | 3 | 1 |                       | 7 | 8 | 0 |
              
Hay 9! configuraciones iniciales pero ojo! porque no todas tienen solución. Tenlo en cuenta al hacer las pruebas. 

### EJERCICIO 1. COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 

In [9]:
class Ocho_Puzzle(Problem):

    #   """Problema a del 8-puzzle.  Los estados serán tuplas de nueve elementos,
    #   permutaciones de los números del 0 al 8 (el 0 es el hueco). Representan la
    #   disposición de las fichas en el tablero, leídas por filas de arriba a
    #   abajo, y dentro de cada fila, de izquierda a derecha. Las cuatro
    #   acciones del problema las representaremos mediante las cadenas:
    #  "Mover hueco arriba", "Mover hueco abajo", "Mover hueco izquierda" y
    #   "Mover hueco derecha", respectivamente."""""

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
       # """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)
    
    def actions(self,estado):
        pos_hueco=estado.index(0) # busco la posicion del 0
        accs=list()
        if pos_hueco not in (0,1,2):
            accs.append("Mover hueco arriba")
        if pos_hueco not in (0,3,6):
            accs.append("Mover hueco a la izquierda")
        if pos_hueco not in (2,5,8):
            accs.append("Mover hueco a la derecha")
        if pos_hueco not in (6,7,8):
            accs.append("Mover hueco abajo")
        
        ### EJERCICIO 1.1. COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
        
        return accs     

    def result(self,estado,accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        if accion == "Mover hueco arriba":
            l[pos_hueco] = l[pos_hueco-3]
            l[pos_hueco-3] = 0
        elif accion == "Mover a la izquierda":
            l[pos_hueco] = l[pos_hueco - 1]
            l[pos_hueco - 1] = 0
        elif accion == "Mover a la derecha":
            l[pos_hueco] = l[pos_hueco + 1]
            l[pos_hueco + 1] = 0
        elif accion == "Mover hueco abajo":
            l[pos_hueco] = l[pos_hueco + 3]
            l[pos_hueco + 3] = 0


       ### EJERCICIO 1.2. COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
      
        
        return tuple(l)
    
    def h(self, node):
        #""" Return the heuristic value for a given state. 
        #return 1
        return manhattan(node)
    
    def coste_de_aplicar_accion(self, estado, accion):
        #"""Hemos incluido está función que devuelve el coste de un único operador (aplicar accion a estado). Por defecto, este
        #coste es 1. Reimplementar si el problema define otro coste """ 
        #Reimplementación de esta funcion. Simplemente se añaden las restricciones
        #pertientes.
        pos_hueco = estado.index(0)
        l = list(estado)
        if pos_hueco in (0,1,2) and accion == "Mover hueco arriba":
            coste = 0
        elif pos_hueco in (0,3,6) and accion == "Mover hueco a la izquierda":
            coste = 0
        elif pos_hueco in (2,5,8) and accion == "Mover hueco a la derecha":
            coste = 0
        elif pos_hueco in (6,7,8) and accion == "Mover hueco abajo":
            coste = 0
        else:
            coste = 1 #Suponemos que mover el hueco a cualquier dirección posible es 1
        return coste

### Ejercicio 1.3. Probar los siguientes ejemplos que se pueden ejecutar una vez se ha definido la clase:

In [10]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [11]:
p8.actions(p8.initial)
#Respuesta: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

['Mover hueco arriba',
 'Mover hueco a la izquierda',
 'Mover hueco a la derecha']

In [12]:
p8.result(p8.initial,"Mover hueco arriba")

(2, 8, 3, 1, 0, 4, 7, 6, 5)

In [13]:
p8.result(p8.initial,"Mover hueco abajo")
#Da error debido a que se invoca cuando el hueco no puede moverse abajo (se sale de rango)
#Esta funcion se debería invocar con el valor devuelto de actions() para asegurarse de que no
#se sale del rango (suponemos que está dentro de la precondición de resul)

IndexError: list index out of range

In [14]:
p8.result(p8.initial,"Mover hueco derecha")

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [15]:
p8.coste_de_aplicar_accion(p8.initial,"Mover hueco abajo")
#Devuelve 0 debido a que se ha definido la función otra vez considerando las restricciones

0

## Parte II: Experimentación con los algoritmos implementados. Ejecución de los algoritmos de búsqueda de soluciones para una instancia del Problema.

### Usaremos búsqueda en anchura y en profundidad para encontrar soluciones tanto al problema de las jarras como al problema del ocho puzzle con distintos estados iniciales.

In [16]:
# Cargamos el módulo con los algoritmos de búsqueda.
from search import *
from search import uniform_cost_search, breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

In [17]:
breadth_first_tree_search(Jarras()).solution()

['llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3',
 'vaciar jarra de 3',
 'trasvasar de jarra de 4 a jarra de 3',
 'llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3']

In [18]:
depth_first_graph_search(Jarras()).solution()

['llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'vaciar jarra de 4',
 'trasvasar de jarra de 3 a jarra de 4']

### Ejercicio 2. Prueba los algoritmos de búsqueda ciega con el puzzle de 8

In [19]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [20]:
p8.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [21]:
#breadth_first_tree_search(EightPuzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))).solution()
# Busqueda en anchura sin control de repetidos.  
# Busqueda en anchura es completo.. debería terminar...  ¿qué crees que está pasando?

In [16]:
%%time
breadth_first_tree_search(Ocho_Puzzle((1,2,3,4,5,6,0,7,8))).solution()
#Lo hemos ejecutado dos veces y se ha quedado "tostado" el pc (después de más de una hora), supondremos que tarda mucho, pero, teóiricamente
#si existe solución, la encuentra y es óptima. Los errores del output son por interrumpir la ejecución, le di sin querer
#y no quería tener el pc "tostado" otra vez

KeyboardInterrupt: 

In [22]:
estado = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))

In [ ]:
%%time
breadth_first_tree_search(estado)#.solution()
# Respuesta: ['UP', 'LEFT', 'UP', 'LEFT', 'DOWN', 'RIGHT', 'RIGHT', 'DOWN']
#Se me ha vuelto a quedar "colgado" el pc

In [48]:
%%time
depth_first_graph_search(estado) #.solution()


CPU times: user 65 µs, sys: 2 µs, total: 67 µs
Wall time: 72.2 µs


In [49]:
%%time
breadth_first_graph_search(estado)#.solution()


CPU times: user 39 µs, sys: 1 µs, total: 40 µs
Wall time: 42.9 µs


#### En este ejercicio se ha podido observar los resultados y tiempo de la ejecución de los algoritmos de búsqueda ciega.  Escribe aquí tus conclusiones:

Debido a que, por algún error, no podemos utilizar la función ".solution()" hablaremos esclusivamente del tiempo.
Como se puede ovservar, la busqueda por anchura, en ambos casos ("graph" y "tree") tarda mucho más en comparación a la de profundida, y, teóricamente, al terminar si hay solución al problema, la encuentra y es óptima, en contraparte que el otro algoritmo (profundidad), que no garantiza una solución óptima
 
**********************************************************************************



### Ejercicio 3: definir al menos las siguientes funciones heurísticas para el 8 puzzle:
* linear(node): cuenta el número de casillas mal colocadas respecto al estado final.
* manhattan(node): suma la distancia Manhattan desde cada casilla a la posición en la que debería estar en el estado final.
* max_heuristic(node: maximo de las dos anteriores
* sqrt_manhattan(node):  raíz cuadrada de la distancia Manhattan

In [22]:
# Heuristicas para el 8 Puzzle 

def linear(node):
    #goal = node.state.goal
    l = list(node.state)
    goal = (1,2,3,4,5,6,7,8,0)
    cont = 0
    for i in range(len(goal)):
        if goal[i] == l[i]:
            cont = cont + 1
    return cont

def manhattan(node):
    state = node.state
    #????
    #Para calcular la distancia manhattan transformo el tablero en una "matriz"
    #calculando las coordenadas x e y a partir de las posiciones, las y-es son
    #los restos de dividir la posición entre tres y las x-es son los cocientes.
    #Tras eso, calculo las casillas que hay entre la posición original y la actual.
    mhd = 0
    l = list(state)
    for i in range(len(l)):
        if l[i] == 0:
            posGoal = 8
        else:
            posGoal = (l[i] - 1)
        yGoal = posGoal % 3
        xGoal = posGoal / 3
        yActual = i / 3
        xActual = i % 3
        yMh = abs(yGoal - yActual)
        xMh = abs(xGoal - xActual)
        mhd += xMh + yMh
    return mhd

def sqrt_manhattan(node):
    return math.sqrt(manhattan(node))
    
    #return math.sqrt(mhd)

def max_heuristic(node):
    score1 = manhattan(node)
    score2 = linear(node)
    return max(score1, score2)
        

### Ejercicio 4. 
Usar las implementaciones de los algoritmos que correspondan a búsqueda_coste_uniforme, busqueda_primero_el_mejor y búsqueda_a_estrella (con las heurísticas anteriores) para resolver el problema del 8 puzzle para el siguiente estado inicial y comparar los costes temporales usando %timeit y comentar los resultados.

              +---+---+---+
              | 2 | 4 | 3 |
              +---+---+---+
              | 1 | 5 | 6 |
              +---+---+---+
              | 7 | 8 | H |
              +---+---+---+


In [23]:
%%timeit
#Coste uniforme
puzle = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0)) 
uniform_cost_search(puzle)

21.4 µs ± 794 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [24]:
%%timeit
#Primero el mejor
best_first_graph_search(puzle, manhattan)

NameError: name 'puzle' is not defined

In [ ]:
puzle.initial

In [ ]:
puzle.goal

In [ ]:
%%timeit
astar_search(puzle, linear)

In [ ]:
%%timeit
astar_search(puzle, manhattan)

In [ ]:
%%timeit
astar_search(puzle, sqrt_manhattan)

In [25]:
%%timeit
astar_search(puzle, max_heuristic)

NameError: name 'puzle' is not defined

¿Has notado diferencias en los tiempos de ejecución? Vamos a medirlo.Aunque las heurísticas no afectan a la solución obtenida sí hay diferencias importantes en el tiempo de cálculo

Se puede observar que, en cuanto a los tiempos de resolución el más rapido ha sido el algoritmo de coste uniforme.
El "primero el mejor" tiene un tiempo medio. En cuanto a los A*, podemos ver que heurísticas ayudan a que el problemas se resuelva más rapido. Como se puede ver la heurísticas
<br>

In [26]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
puzzle_2 = Ocho_Puzzle((1, 2, 3, 4, 5, 6, 0, 7, 8))
puzzle_3 = Ocho_Puzzle((1, 2, 3, 4, 5, 7, 8, 6, 0))

In [27]:
%%timeit
astar_search(puzzle_1, linear)
astar_search(puzzle_2, linear)
astar_search(puzzle_3, linear)

78.7 µs ± 1.58 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [28]:
%%timeit
astar_search(puzzle_1, manhattan)
astar_search(puzzle_2, manhattan)
astar_search(puzzle_3, manhattan)

104 µs ± 959 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [29]:
%%timeit
astar_search(puzzle_1, sqrt_manhattan)
astar_search(puzzle_2, sqrt_manhattan)
astar_search(puzzle_3, sqrt_manhattan)

101 µs ± 733 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [30]:
%%timeit
astar_search(puzzle_1, max_heuristic)
astar_search(puzzle_2, max_heuristic)
astar_search(puzzle_3, max_heuristic)

119 µs ± 3.27 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


#### Escribe aquí tus conclusiones sobre qué heurística es mejor y por qué.

# ----------------------------------------------------------------------------
# ----------------------------------------------------------------------------


## Parte III:  Calcular estadísticas sobre la ejecución de los algoritmos para resolución de problemas de ocho puzzle. 
### El objetivo es comprobar experimentalmente las propiedades teóricas de los algoritmos vistas en clase.
Usaremos la función %timeit para medir los tiempos y para el espacio una version modificada de Problema que almacena el número de nodos.



En la clase modificada también vamos a incluir el calculo que nos diga si un cierto tablero tiene solución o no. Esto es muy útil.. como hemos comentado al principio solo algunos tableros tienen solucion.
En el caso del puzle de 8 un tablero se ha demostrado que se puede comprobar calculando el número de inversiones. Si es impar el tablero no tiene solución. 

The solvability of a configuration can be checked by calculating the Inversion Permutation. If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.

In [36]:
# Hacemos una definición ampliada de la clase Problem de AIMA que nos va a permitir experimentar con distintos
# estados iniciales, algoritmos y heurísticas, para resolver el 8-puzzle. 
# Añadimos en la clase ampliada la capacidad para contar el número de nodos analizados durante la
# búsqueda:

class Problema_con_Analizados(Problem):

    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda.""" 
         
    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.analizados  = 0
        self.goal = problem.goal

    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion)

    def goal_test(self, estado):
        self.analizados += 1
        return self.problem.goal_test(estado)

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problem.coste_de_aplicar_accion(estado,accion)
    
    def check_solvability(self, state):
        """ Checks if the given state is solvable """

        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1
        
        return inversion % 2 == 0        

In [37]:
estado_inicial = (1,2,3,4,5,6,7,0,8)
p8p=Problema_con_Analizados(Ocho_Puzzle(estado_inicial))
p8 = Ocho_Puzzle(estado_inicial)

In [38]:
p8p.initial

(1, 2, 3, 4, 5, 6, 7, 0, 8)

In [39]:
p8p.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [43]:
%%timeit
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
#astar_search(puzzle_1, h2_ocho_puzzle).solution()
astar_search(puzzle_1, manhattan)

34.5 µs ± 369 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [46]:
%%timeit
astar_search(p8, linear)

31.1 µs ± 457 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [47]:
%%timeit
astar_search(p8p, max_heuristic)

46.7 µs ± 778 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [49]:
def resuelve_ocho_puzzle(estado_inicial, algoritmo, h=None):
    """Función para aplicar un algoritmo de búsqueda dado al problema del ocho
       puzzle, con un estado inicial dado y (cuando el algoritmo lo necesite)
       una heurística dada.
       Ejemplo de uso:
           puzzle_1 = (2, 4, 3, 1, 5, 6, 7, 8, 0)
           resuelve_ocho_puzzle(puzzle_1,astar_search,h2_ocho_puzzle)
        Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 
        'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
        Algoritmo: astar_search
        Heurística: h2_ocho_puzzle
        Longitud de la solución: 8. Nodos analizados: 11
       """

    p8p=Problema_con_Analizados(Ocho_Puzzle(estado_inicial))
    if p8p.check_solvability(estado_inicial):
        if h: 
            sol= algoritmo(p8p,h).solution()
        else: 
            sol= algoritmo(p8p).solution()
        print("Solución: {0}".format(sol))
        print("Algoritmo: {0}".format(algoritmo.__name__))
        if h: 
            print("Heurística: {0}".format(h.__name__))
        else:
            pass
        print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p8p.analizados))
    else: 
        print("Este problema no tiene solucion. ")


In [50]:
resuelve_ocho_puzzle(estado_inicial,astar_search,manhattan)

AttributeError: 'NoneType' object has no attribute 'solution'

#### Ejercicio 5

#Intentar resolver usando las distintas búsquedas y en su caso, las distintas
#heurísticas, el problema del 8 puzzle para los siguientes estados iniciales:

          E1              E2              E3              E4
           
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    
     | 2 | 8 | 3 |   | 4 | 8 | 1 |   | 2 | 1 | 6 |   | 5 | 2 | 3 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+
     | 1 | 6 | 4 |   | 3 | H | 2 |   | 4 | H | 8 |   | H | 4 | 8 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+
     | 7 | H | 5 |   | 7 | 6 | 5 |   | 7 | 5 | 3 |   | 7 | 6 | 1 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+
   
 Se pide, en cada caso, obtener detalles del tiempo y espacio necesario para la resolución de estos estados.
 Hacerlo con la función resuelve_ocho_puzzle, para
 obtener, además de la solución, la longitud (el coste) de la solución
 obtenida y el número de nodos analizados. Anotar los resultados en la
 siguiente tabla (L, longitud de la solución, NA, nodos analizados, T, tiempo, y
 justificarlos con las distintas propiedades teóricas estudiadas.

-----------------------------------------------------------------------------------------
                                       E1           E2           E3          E4
    Anchura                            L=           L=           L=          L=
                                       T=           T=           T=          T=  
                                       NA=          NA=          NA=         NA= 
   
    Profundidad                        L=           L=           L=          L=  
                                       T=           T=           T=          T= 
                                       NA=          NA=          NA=         NA= 
                                       
    Coste Uniforme                     L=           L=           L=          L=
                                       T=           T=           T=          T=  
                                       NA=          NA=          NA=         NA= 
                                       
    Primero el mejor (linear)          L=           L=           L=          L=
                                       T=           T=           T=          T=  
                                       NA=          NA=          NA=         NA=
                                                                            
    Primero el mejor (manhattan)       L=           L=           L=          L= 
                                       T=           T=           T=          T=  
                                       NA=          NA=          NA=         NA=
                                                                             
    A* (linear)                        L=           L=           L=          L= 
                                       T=           T=           T=          T=  
                                       NA=          NA=          NA=         NA=
                                                                             
     A* (manhattan)                    L=           L=           L=          L= 
                                       T=           T=           T=          T=  
                                       NA=          NA=          NA=         NA=

 -----------------------------------------------------------------------------------------




### Ejercicio 6: [OPCIONAL] Definir una heurística más informada y completa la tabla anterior para ver cómo afecta al número de nodos generados por los algoritmos

In [79]:
# Proponemmos como heurística h3 la se obtiene sumando a la heurística
# manhattan una componente que cuantifica la "secuencialidad" en las
# casillas de un tablero, al recorrerlo en el sentido de las aguas del reloj
# ¿Es h3 admisble? Comprobar cómo se comporta esta heurística cuando se usa en
# A*, con cada uno de los estados anteriores. Comentar los resultados.
# Puedes proponer otra heuristica siempre que sea más informada que todas las heurísticas ya utilizadas.

         

# ---------------------------------------------------------------------------

